In [1]:
portfolio_sd_t <- function(sd, cov, weight){
  stopifnot(length(sd) > 1, is.matrix(cov), length(sd) == length(weight))
  for(i in 1:(length(sd)-1)){
    for(j in (i+1):length(sd)){
      n <- sum(weight[i:length(sd)]^2*sd[i:length(sd)]^2)
      m <- sum(2*weight[i]*weight[j]*cov[i,j])
      sd <- sqrt(sum(weight[i:length(sd)]^2*sd[i:length(sd)]^2+
                       2*weight[i]*weight[j]*cov[i,j]))
      return(sd)
    }
  }
}
portfolio_return <- function(returns, weight){
  if(!is.vector(returns) > 1){
    mean <- apply(returns,2, mean)
    }else{
    if(is.vector(returns)){
      mean <- returns
    }
  }
  total_return <- sum(weight*mean)
  return(total_return)
}

In [2]:
pacotes <- c("tidyquant", "tidyr", "dplyr","stats")
options(warn = -1)
for(i in pacotes){
    suppressPackageStartupMessages(library(i, character.only = T))
}

In [3]:
acoes <- c("TRPL4.SA","ITSA4.SA","ITUB4.SA","EGIE3.SA","CPLE3.SA",
           "LREN3.SA","HYPE3.SA","B3SA3.SA","VVAR3.SA","ABCB4.SA",
          "SUZB3.SA","BEES3.SA","ABEV3.SA","FLRY3.SA")
retornos_d <- tq_get(acoes, from = Sys.Date()-365) %>%
  group_by(symbol) %>%
  tq_transmute(select = close, mutate_fun= periodReturn, period = "daily")
retornos_a <- tq_get(acoes, from = Sys.Date()-365*5) %>%
  group_by(symbol) %>%
  tq_transmute(select = close, mutate_fun= periodReturn, period = "yearly") %>%
  mutate(year = year(date)) %>%
  group_by(year,symbol) %>%
  summarise(yearly.returns) %>%
  spread(key = "symbol", value = "yearly.returns")

`summarise()` regrouping output by 'year' (override with `.groups` argument)



In [4]:
sd_d <- retornos_d %>%
  group_by(symbol) %>%
  summarise(sd = sd(daily.returns)) %>%
  spread(key = "symbol", value = sd)
cov <- retornos_d %>%
  spread(key = "symbol", value = daily.returns)
cov$date <- NULL
cov <- cov(cov)

`summarise()` ungrouping output (override with `.groups` argument)



In [5]:
pesos <- c(0.06,0.02,0.06,0.04,0.12,0.13,0.02,0.07,0.13,0.03,0.12,0.04,0.1,0.06)

In [6]:
sd_cart <- portfolio_sd_t(sd = sd_d, cov = cov, weight = pesos)*sqrt(252)
retorno_esp <- portfolio_return(returns = retornos_a[2:ncol(retornos_a)], weight = pesos)
paste0("Desvio-Anual =", round(sd_cart*100,3), "%  Retorno Esperado =", round(retorno_esp*100,3),"%")
sharpe <- (retorno_esp-0.0225)/sd_cart
names(sharpe) <- "Sharpe da Carteira"
round(sharpe,3)

[1] "Desvio-Anual =16.835%  Retorno Esperado =29.112%"

Sharpe da Carteira 
             1.596

In [7]:
sd_cart
retorno_esp

[1] 0.1683456

[1] 0.2911243

In [9]:
#Minha carteira tem beta de 0.8239262828